In [1]:
import folium
import pandas as pd

In [2]:
names = ['center']
df_list = []
for n in names:
    path = 'zahyo/{}.csv'.format(n)
    df_list.append(pd.read_csv(path, names=['name', 'address', 'age', 'rent', 'kanri', 'sikikin', 'reikin', 'kinds', 'area', 'floor', 'x', 'y']))

df = pd.concat(df_list)

In [3]:
df.head()

,name,address,age,rent,kanri,sikikin,reikin,kinds,area,floor,x,y
0,地下鉄東西線 バスセンター前駅 4階建 築4年,北海道札幌市中央区北四条東４,築4年,8万円,6000円,-,8万円,2LDK,57.15m2,4階,43.068026,141.361304
1,Ｒａｆｆｉｎｅ近代美術館前,北海道札幌市中央区北一条西１８,築4年,8万円,4000円,-,-,2LDK,52.64m2,2階,43.059626,141.328823
2,Ｎａｋａｊｉｍａ Ｐａｒｋ Ｓｉｄｅ（旧アンジュ・ソワール）,北海道札幌市中央区南十一条西７,築8年,8.6万円,4000円,8.6万円,-,2LDK,53.38m2,2階,43.044540,141.334281
3,Ｎａｋａｊｉｍａ Ｐａｒｋ Ｓｉｄｅ（旧アンジュ・ソワール）,北海道札幌市中央区南十一条西７,築8年,8.6万円,4000円,8.6万円,-,2LDK,53.38m2,4階,43.044540,141.334281
4,地下鉄南北線 中島公園駅 5階建 築8年,北海道札幌市中央区南十一条西７,築8年,8.6万円,4000円,8.6万円,-,2LDK,53.38m2,2階,43.044540,141.334281


In [4]:
#家賃を数値に変換
df['rent'] = df['rent'].str.strip('万円').astype(float)

#同じaddressの処理
address = df['address'].unique()
new_df = []
for adr in address:
    df_adr = df.loc[df['address']==adr]
    value = df_adr['rent'].mean()
    new_df.append([value, df_adr.iloc[0, 10], df_adr.iloc[0,11]])
df = pd.DataFrame(new_df, columns=['rent', 'x', 'y'])

In [5]:
df.head()

,rent,x,y
0,8.00,43.068026,141.361304
1,8.00,43.059626,141.328823
2,8.60,43.044540,141.334281
3,8.76,43.054713,141.359370
4,7.70,43.066099,141.368319


In [6]:
#color決定
colors = ['#ffd5d5', '#ff5555', '#d50000', '#550000']
num_color = len(colors)
df.loc[df['rent']<df['rent'].quantile(1/num_color), 'color'] = colors[0]
for i in range(1, num_color-1):
    df.loc[(df['rent'].quantile(i/num_color) <= df['rent']) & (df['rent'] < df['rent'].quantile((i+1)/num_color)), 'color'] = colors[i]
df.loc[df['rent']>=df['rent'].quantile((num_color-1)/num_color), 'color'] = colors[-1]

In [7]:
location = [df['x'].mean(), df['y'].mean()]
_map = folium.Map(location=location, zoom_start=12, tiles="Stamen Toner")
for i in range(len(df)):
    folium.Circle(radius=100, location=[df.loc[i, 'x'], df.loc[i, 'y']], color=df.loc[i, 'color'], fill_color=df.loc[i, 'color'], fill=True,).add_to(_map)

print('{}% : - {:.2f}'.format(int((1)/num_color*100), df['rent'].quantile((1)/num_color)))
for i in range(1, num_color):
    print('{}% : {:.2f} - {:.2f}'.format(int((i+1)/num_color*100), df['rent'].quantile((i)/num_color), df['rent'].quantile((i+1)/num_color)))
_map

25% : - 8.00
50% : 8.00 - 8.55
75% : 8.55 - 8.89
100% : 8.89 - 9.60
